# Stock-Selling Strategy Using Leap Hybrid CQM Solver

This example finds a stock-selling strategy for a simplified market model to demonstrate using a [Leap](https://cloud.dwavesys.com/leap/) hybrid [CQM](https://docs.ocean.dwavesys.com/en/latest/concepts/index.html#term-CQM) solver on a constrained problem with integer and binary variables.

In this very simple market, you have some number of shares that you want to sell in daily blocks over a defined interval of days. Each sale of shares affects the market price of the stock,

$$
p_i = p_{i-1} + \alpha s_{i-1},
$$

where $p_i$ and $s_i$ are, respectively, the price and the number of shares sold on day $i$, and $\alpha$ is some multiplier.

The goal of this problem is to find the optimal number of shares to sell per day to maximize revenue from the total sales.

The [market with taxation](https://docs.ocean.dwavesys.com/en/latest/examples/hybrid_cqm_stock_selling.html#example-cqm-stock-tax) section adds a tax to the market model to demonstrate the incorporation of binary variables into the CQM.


In [ ]:
! pip install dwave-ocean-sdk

In [21]:
! dwave auth login --oob

Please visit the following URL to authorize Ocean: https://cloud.dwavesys.com/leap/openid/authorize?response_type=code&client_id=805325&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+get_token&state=5Aw5nWysCszUTbTFnoqBc6mqO7h2SJ&code_challenge=SWpUfkSPVdeld1DRkV3DFfNk3U-Jdgp-NSXLu5sMPhE&code_challenge_method=S256

Authorization code: argxQWfJbpKjHJfxZdWGJ8mDLesk0L

Authorization completed successfully. You can now use "dwave auth get" to fetch your token.


In [22]:
! dwave auth get

Access token: xvkOivGgy73qiRCWPlYh04vUfQxWMi
Expires at: 2024-06-28T06:03:14Z (timestamp=1719554594) (valid)


## Example Requirements

The code in this example requires that your development environment have [Ocean software](https://github.com/dwavesystems/dwave-ocean-sdk) and be configured to access SAPI, as described in the [Initial Set Up](https://docs.ocean.dwavesys.com/en/latest/getting_started.html#gs-initial-setup) section.

## Solution Steps

Section [Workflow Steps: Formulation and Sampling](https://docs.ocean.dwavesys.com/en/latest/overview/solving_problems.html#solving-problems) describes the problem-solving workflow as consisting of two main steps: (1) Formulate the problem as an [objective function](https://docs.ocean.dwavesys.com/en/latest/concepts/index.html#term-Objective-function) in a supported model and (2) Solve your model with a D-Wave [solver](https://docs.ocean.dwavesys.com/en/latest/concepts/index.html#term-Solver).



## Formulate the Problem

First, define the market parameters.

- `max_days` is the period over which you should sell all your shares.
- `total_shares` is the number of shares you own (equal to
).
- `price_day_0` is the stock price on the first day of the period.
- `alpha` is a multiplier, $\sum_i s_i$, that controls how much the stock price increases for each share sold into the market.

In [6]:
max_days = 10
total_shares = 100
price_day_0 = 50
alpha = 1

Instantiate a CQM.

In [7]:
from dimod import ConstrainedQuadraticModel
cqm = ConstrainedQuadraticModel()

## Objective Function

The objective function to maximize is the revenue from selling shares. Because you own an integer number of shares, it is convenient to use integer variables to indicate the number of shares sold each day, `shares`. For simplicity, this model assumes stock prices, `price`, are also integers [1](https://docs.ocean.dwavesys.com/en/latest/examples/hybrid_cqm_stock_selling.html#id2).

[1]: One could use integer variables to model stock prices in dollars and cents by multiplying the values by 100; however, this greatly increases the search space. One could also create a compromise model with somewhat greater resolution and search space by rounding to the nearest dime and multiplying prices by 10, for example.

Bounds on the range of values for integer variables shrink the solution space the solver must search, so it is helpful to set such bounds; for many problems, you can find bounds from your knowledge of the problem. In this case,

- On any day, you cannot sell more than the total number of shares you start with.
- The maximum share price is the sum of the initial price and the total price increase that would result from selling all your shares,

$$\max(p) = p_0 + \alpha \star \sum_i s_i$$

In [8]:
from dimod import Integer
max_p = price_day_0 + alpha*total_shares
shares = [Integer(f's_{i}', upper_bound=total_shares) for i in range(max_days)]
price = [Integer(f'p_{i}', upper_bound=max_p) for i in range(max_days)]

Daily revenue is the number of shares sold multiplied by the price on each sales day.

In [10]:
revenue = [s*p for s, p in zip(shares, price)]
print(revenue)

[QuadraticModel({'s_0': 0.0, 'p_0': 0.0}, {('p_0', 's_0'): 1.0}, 0.0, {'s_0': 'INTEGER', 'p_0': 'INTEGER'}, dtype='float64'), QuadraticModel({'s_1': 0.0, 'p_1': 0.0}, {('p_1', 's_1'): 1.0}, 0.0, {'s_1': 'INTEGER', 'p_1': 'INTEGER'}, dtype='float64'), QuadraticModel({'s_2': 0.0, 'p_2': 0.0}, {('p_2', 's_2'): 1.0}, 0.0, {'s_2': 'INTEGER', 'p_2': 'INTEGER'}, dtype='float64'), QuadraticModel({'s_3': 0.0, 'p_3': 0.0}, {('p_3', 's_3'): 1.0}, 0.0, {'s_3': 'INTEGER', 'p_3': 'INTEGER'}, dtype='float64'), QuadraticModel({'s_4': 0.0, 'p_4': 0.0}, {('p_4', 's_4'): 1.0}, 0.0, {'s_4': 'INTEGER', 'p_4': 'INTEGER'}, dtype='float64'), QuadraticModel({'s_5': 0.0, 'p_5': 0.0}, {('p_5', 's_5'): 1.0}, 0.0, {'s_5': 'INTEGER', 'p_5': 'INTEGER'}, dtype='float64'), QuadraticModel({'s_6': 0.0, 'p_6': 0.0}, {('p_6', 's_6'): 1.0}, 0.0, {'s_6': 'INTEGER', 'p_6': 'INTEGER'}, dtype='float64'), QuadraticModel({'s_7': 0.0, 'p_7': 0.0}, {('p_7', 's_7'): 1.0}, 0.0, {'s_7': 'INTEGER', 'p_7': 'INTEGER'}, dtype='float64'),

In [11]:
cqm.set_objective(-sum(revenue))

## Constraints

The simplified market in this problem has the following constraints:

1. In total you can sell only the number of shares you own, no more, $\sum_i s_i \le$ `total_shares`.

In [12]:
cqm.add_constraint(sum(shares) <= total_shares, label='Sell only shares you own')

'Sell only shares you own'

2. On the first day of the selling period, the stock has a particular price $p_0=$ `price_day_0`

In [13]:
cqm.add_constraint(price[0] == price_day_0, label='Initial share price')

'Initial share price'

3. The stock price increases in proportion to the number of shares sold the previous day: $p_i = p_{i-1}+\alpha s_{i-1}$

In [14]:
print(max_days)

10


In [15]:
for i in range(1, max_days):
   pricing = cqm.add_constraint(price[i] - price[i-1] - alpha*shares[i-1] == 0, label=f'Sell at the price on day {i}')
   print(pricing)

Sell at the price on day 1
Sell at the price on day 2
Sell at the price on day 3
Sell at the price on day 4
Sell at the price on day 5
Sell at the price on day 6
Sell at the price on day 7
Sell at the price on day 8
Sell at the price on day 9


For a sales period of ten days, this CQM has altogether 11 constraints:

In [16]:
len(cqm.constraints)

11

## Solve the Problem by Sampling

Instantiate a [`LeapHybridCQMSampler`](https://docs.ocean.dwavesys.com/en/latest/docs_system/reference/samplers.html#dwave.system.samplers.LeapHybridCQMSampler) class sampler,

Note: Use the [instructions](https://docs.ocean.dwavesys.com/en/latest/overview/sapi.html) to find the API Token.

In [35]:
from google.colab import userdata
DWAVE_API_TOKEN = userdata.get('DWAVE_API_TOKEN')

In [36]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler(token=DWAVE_API_TOKEN)

In [37]:
sampler

In [38]:
sampler.solver.name

'hybrid_constrained_quadratic_model_version1'

For one particular execution, with a maximum allowed runtime of a minute, the CQM hybrid solver returned 41 samples, out of which 24 were solutions that met all the constraints:

In [39]:
sampleset = sampler.sample_cqm(
    cqm,
    time_limit=60,
    label="SDK Examples - Stock-Selling Strategy")

print("{} feasible solutions of {}.".format(sampleset.record.is_feasible.sum(), len(sampleset)))

117 feasible solutions of 150.


The small function below extracts from the returned sampleset the best feasible solution and parses it.

In [40]:
def parse_best(sampleset):
   best = sampleset.filter(lambda row: row.is_feasible).first
   s = [val for key, val in best.sample.items() if "s_" in key]
   p = [val for key, val in best.sample.items() if "p_" in key]
   r = [p*s for p, s in zip(p, s)]
   return r, s, best

Parse and print the best feasible solution:

In [41]:
r, s, _ = parse_best(sampleset)
print("Revenue of {} found for daily sales of: \n{}".format(sum(r), s))

Revenue of 9499.0 found for daily sales of: 
[10.0, 10.0, 9.0, 10.0, 10.0, 10.0, 11.0, 10.0, 10.0, 10.0]


## Market with Taxation

The previous sections made use of only integer variables. Quadratic models also accept binary variables. This section models a market in which you pay an additional tax on early sales and uses a binary variable to incorporate that update into the CQM created in the previous sections.

Consider a market in which you pay a tax in amount, `tax_payment`, for selling shares during the first `taxed_period` days of the period in which you can sell your shares.

In [42]:
taxed_period = 3
tax_payment = 225

Because you either pay this tax or do not pay it, you can use a binary variable, `t`, to indicate payment. You can update the previous objective by reducing the revenue from share sales by the tax payment (adding it to the negative revenue) if the `t` binary variable is 1:

In [43]:
from dimod import Binary
t = Binary('t')
cqm.set_objective(tax_payment*t - sum(revenue))

In [44]:
print(t)

BinaryQuadraticModel({'t': 1.0}, {}, 0.0, 'BINARY')


Binary variable `t` should be `True` if sales in the first `taxsed_period` days of the period are greater than zero; otherwise it should be `False`:

$$\sum_{i<\text{taxed_period}} s_i > 0 \rightarrow t=1$$

$$\sum_{i<\text{taxed_period}} s_i > 0 \rightarrow t=0$$

One way to set such an indicator variable is to create a pair of linear constraints:

$$
	\frac{\sum_{i < \text{taxed_period}} s_i}{\sum_i s_i} \le t \le \sum_{i < \text{taxed_period}} s_i
$$

To show that this pair of inequalities indeed sets the desired binary indicator, the table below shows, **bolded**, the binary values $t$ must take to simultaneously meet both inequalities for $\sum_{i<\text{taxed_period}}s_i$ with sample values 0, 1, and 5 for the previous configured `total_shares = 100`.

| $\frac{\sum_{i < \text{taxed_period}} s_i}{\sum_i s_i}$ | $\sum_{i < \text{taxed_period}} s_i$ | $\pmb{t}$ | $\frac{\sum_{i < \text{taxed_period}} s_i}{\sum_i s_i} \le \pmb{t} \le \sum_{i < \text{taxed_period}} s_i$                |
|-----------------------------------------------------------|----------------------------------------|-------------|-------------------------------------------------------------------------------------------------------------------------|
| 0                                                         | 0                                      | $\pmb{0}$  | $0 = \pmb{0} = 0$                                                                                                      |
| $\frac{1}{100}$                                         | 1                                      | $\pmb{1}$  | $\frac{1}{100} < \pmb{1} = 1$                                                                                          |
| $\frac{5}{100}$                                         | 5                                      | $\pmb{1}$  | $\frac{5}{100} < \pmb{1} < 5$                                                                                          |


Add these two constraints to the previously created CQM:

In [45]:
cqm.add_constraint(t - sum(shares[:taxed_period]) <= 0, label="Tax part 1")
cqm.add_constraint(1/total_shares*sum(shares[:taxed_period]) - t <= 0, label="Tax part 2")

'Tax part 2'

Submit the CQM to the selected solver. For one particular execution, with a maximum allowed runtime of a minute, the CQM hybrid sampler returned 50 samples, out of which 33 were solutions that met all the constraints:

In [46]:
sampleset = sampler.sample_cqm(
    cqm,
    time_limit=60,
    label="SDK Examples - Stock-Selling Strategy")

print("{} feasible solutions of {}.".format(sampleset.record.is_feasible.sum(), len(sampleset)))

124 feasible solutions of 149.


Parse and print the best feasible solution:

In [47]:
r, s, best = parse_best(sampleset)
income = sum(r) - best.sample['t']*tax_payment
print("Post-tax income of {} found for daily sales of: \n{}".format(income, s))

Post-tax income of 9285.0 found for daily sales of: 
[0.0, 0.0, 0.0, 15.0, 14.0, 14.0, 14.0, 15.0, 14.0, 14.0]


Notice that the existence of this tax, though avoided in the sales strategy found above, has reduced your income by a little less than the tax fee (the maximum income if you had paid the tax would be 9275). If the tax is slightly reduced, it is more profitable to sell during the taxation period and pay the tax:

In [ ]:
tax_payment = 220
cqm.set_objective(tax_payment*t - sum(revenue))
sampleset = sampler.sample_cqm(
    cqm,
    time_limit=60,
    label="SDK Examples - Stock-Selling Strategy")
r, s, best = parse_best(sampleset)
income = sum(r) - best.sample['t']*tax_payment
print("Post-tax income of {} found for daily sales of: \n{}".format(income, s))